In [5]:
import os
#from tqdm import tqdm
#from tqdm.notebook import tqdm
from PIL import Image
import numpy as np

import torch

import torch.nn.functional as F
import torchvision.transforms as transforms

from data.base_dataset import Normalize_image
from utils.saving_utils import load_checkpoint_mgpu

from networks import U2NET
import requests
import extcolors
import json
from collections import OrderedDict
import csv

In [8]:
directory = 'put image directory'
files = os.listdir(directory)
device = 'cuda'
checkpoint_path = 'cloth_segm_u2net_latest.pth'

In [9]:
def get_palette(num_cls):
    """ Returns the color map for visualizing the segmentation mask.
    Args:
        num_cls: Number of classes
    Returns:
        The color map
    """
    n = num_cls
    palette = [0] * (n * 3)
    for j in range(0, n):
        lab = j
        palette[j * 3 + 0] = 0
        palette[j * 3 + 1] = 0
        palette[j * 3 + 2] = 0
        i = 0
        while lab:
            palette[j * 3 + 0] |= (((lab >> 0) & 1) << (7 - i))
            palette[j * 3 + 1] |= (((lab >> 1) & 1) << (7 - i))
            palette[j * 3 + 2] |= (((lab >> 2) & 1) << (7 - i))
            i += 1
            lab >>= 3
    return palette

In [ ]:
#save color result in csv
transforms_list = []
transforms_list += [transforms.ToTensor()]
transforms_list += [Normalize_image(0.5, 0.5)]
transform_rgb = transforms.Compose(transforms_list)

net = U2NET(in_ch=3, out_ch=4)
net = load_checkpoint_mgpu(net, checkpoint_path)
net = net.to(device)
net = net.eval()

palette = get_palette(4)

f = open('./result.csv','a', newline='')
wr = csv.writer(f)
z = 0

def run():
    for j,i in enumerate(files):

        print(j)
        try:
            img = Image.open("./fw_2022/{}".format(i))
            img_size = img.size
        except:
            print("error3")
            continue
        try:
            re_img = img.resize((768, 768), Image.BICUBIC)
            image_tensor = transform_rgb(re_img)
            image_tensor = torch.unsqueeze(image_tensor, 0)
            output_tensor = net(image_tensor.to(device))
            output_tensor = F.log_softmax(output_tensor[0], dim=1)
            output_tensor = torch.max(output_tensor, dim=1, keepdim=True)[1]
            output_tensor = torch.squeeze(output_tensor, dim=0)
            output_tensor = torch.squeeze(output_tensor, dim=0)
            output_arr = output_tensor.cpu().numpy()
        except:
            print("error1")
            continue
        output_img = Image.fromarray(output_arr.astype('uint8'), mode='L')
        output_img = output_img.resize(img_size, Image.BICUBIC)
        
        output_img.putpalette(palette)
        
        img_arr = np.array(output_img.resize((500,700)))
        mask = img_arr > 0
        image_array = np.array(img.resize((500,700)))
        try:
            PIL_image = Image.fromarray(image_array[mask].reshape(1,-1,3)).convert('RGB')
        except:
            print("error2")
            continue
        colors, pixel_count = extcolors.extract_from_image(PIL_image)
        color_dict = OrderedDict()
        for c in colors:
            color_dict["{},{},{}".format(c[0][0],c[0][1],c[0][2])] = round((c[1] / pixel_count) * 100, 2)
        color_rgb = json.dumps(color_dict)
        wr.writerow([i,color_rgb])
    f.close()

In [ ]:
run()

In [ ]:
#csv to dict
color_dict = {}
color_dict["result"] = []
for i in colordf.iloc:
    small_dict = {}
    small_dict["name"] = i["image_name"]
    small_dict["RGB"] = eval(i["color_result"])
    color_dict["result"].append(small_dict)
with open('./color_result.json'.format(file_name), 'a', encoding="utf-8") as make_file:
    json.dump(color_dict, make_file, ensure_ascii=False, indent="\t")